In [ ]:
import xmlrpc.client 
import pandas as pd

# Conexión con Odoo
username = "juan.cano@donsson.com"  # tu usuario
password = "1000285668"         # tu contraseña
url = "https://donsson.com"     # URL del servidor
db = "Donsson_produccion" # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")




fields_info = models.execute_kw(
    db, uid, password,
    'sale.order',  # Nombre técnico del modelo
    'fields_get',
    []
)

# Convertir a DataFrame para visualizar mejor
df_campos = pd.DataFrame.from_dict(fields_info, orient='index')
df_campos

# TOTAL FACTURAS 

In [16]:
import xmlrpc.client
from datetime import date, timedelta, datetime
import pandas as pd

# Conexión con Odoo (manteniendo tus credenciales)
username = "juan.cano@donsson.com"  # tu usuario
password = "1000285668"         # tu contraseña
url = "https://donsson.com"     # URL del servidor
db = "Donsson_produccion" # nombre de la base de datos


# --- Autenticación ---
common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# --- Fechas ---
weeks = 15

# --- Fechas automáticas ---
hoy = date.today()
fecha_fin = hoy.strftime("%Y-%m-%d")
fecha_inicio = (hoy - timedelta(weeks=weeks)).strftime("%Y-%m-%d")

# --- 1) Buscar facturas válidas (account.invoice) ---

invoice_domain = [
    ("date_invoice", ">=", fecha_inicio),
    ("date_invoice", "<=", fecha_fin),
    ("type", "=", "out_invoice"),    # solo ventas
    ("state", "in", ["open", "paid"])
]

invoice_ids = models.execute_kw(
    db, uid, password,
    "account.invoice", "search",
    [invoice_domain]
)
print(f"Facturas encontradas: {len(invoice_ids)}")

# --- 2) Descargar las líneas de esas facturas (account.invoice.line) ---

# Campos de la LÍNEA de factura. Eliminamos 'number', 'user_id', 'section_id', 'partner_id' porque irán en la factura.
line_fields = ["product_id", "quantity", "price_subtotal", "invoice_id","create_date","origin"] 

records = []
limit = 20000
offset = 0

while True:
    result = models.execute_kw(
        db, uid, password,
        "account.invoice.line", "search_read",
        [[("invoice_id", "in", invoice_ids)]],
        {"fields": line_fields, "limit": limit, "offset": offset}
    )
    if not result:
        break
    records.extend(result)
    offset += limit
    print(f"Descargados {len(records)} registros de líneas...")

# --- 3) Pasar a DataFrame de líneas ---
line_df = pd.DataFrame(records).fillna(0)


# Separar product_id
line_df["product_id_num"] = line_df["product_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
line_df["product_name"] = line_df["product_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Separar invoice_id
line_df["invoice_id_num"] = line_df["invoice_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
line_df["invoice_name"] = line_df["invoice_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Convertir fecha a datetime
line_df["date_invoice"] = pd.to_datetime(line_df["create_date"], errors="coerce")

# Eliminar las columnas originales problemáticas
line_df = line_df.drop(columns=["invoice_id","create_date"])

print(f"Total de líneas descargadas: {len(line_df)}")

# ----------------------------------------------------
# --- 4) Descargar los campos adicionales de Factura (account.invoice) ---
# ----------------------------------------------------
# Añadimos los campos que quieres: number, user_id, section_id, y también partner_id y store_id
invoice_fields = ["id", "store_id", "number", "user_id", "section_id", "partner_id"]
invoices = models.execute_kw(
    db, uid, password,
    "account.invoice", "read",
    [invoice_ids], # Solo las facturas que encontramos
    {"fields": invoice_fields}
)
invoice_df = pd.DataFrame(invoices)

# --- 5) Procesar campos de la factura ---

# Separar store_id
invoice_df["store_name"] = invoice_df["store_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Separar user_id (Vendedor)
invoice_df["salesperson_name"] = invoice_df["user_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else None
)

# Separar section_id (Equipo de Ventas)
invoice_df["sales_team_name"] = invoice_df["section_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else None
)

# Separar partner_id (Cliente/Partner)
invoice_df["partner_id_num"] = invoice_df["partner_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
# El nombre del partner es el segundo elemento de la tupla (si existe)
invoice_df["client_name_inv"] = invoice_df["partner_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else None
)


# Eliminar columnas originales no deseadas o ya procesadas
invoice_df = invoice_df.drop(columns=["store_id", "user_id", "section_id", "partner_id"])


# ----------------------------------------------------
# --- 6) Fusionar DataFrames ---
# ----------------------------------------------------

# Fusionamos las líneas de factura (line_df) con los datos de las facturas (invoice_df)
df_merged = line_df.merge(
    invoice_df, 
    left_on="invoice_id_num", 
    right_on="id", 
    how="left"
)

# Limpieza final de columnas de IDs de factura
df_merged = df_merged.drop(columns=["invoice_id_num", "product_id_num"])


df_merged['origin'] = df_merged['origin'].astype('string')

# Guardar
#df_merged.to_parquet(f"ventashistoricas{weeks}semanas.parquet", index=False)

print(f"Total de líneas descargadas con información de factura: {len(df_merged)}")

#para 52 semanas se demora aproximadamente 6.5 minutos
#para 56 semanas se demora aproximadamente 7 minutos

Facturas encontradas: 15625
Descargados 20000 registros de líneas...
Descargados 40000 registros de líneas...
Descargados 60000 registros de líneas...
Descargados 60313 registros de líneas...
Total de líneas descargadas: 60313
Total de líneas descargadas con información de factura: 60313


In [17]:
print(f"Fecha primera factura",df_merged["date_invoice"].min())
print(f"Fecha ultima factura",df_merged["date_invoice"].max())

Fecha primera factura 2025-06-18 12:10:00
Fecha ultima factura 2025-10-01 21:20:24


In [39]:
pd.set_option("display.max_colwidth", None)  # muestra toda la celda completa
df_merged.sample(5)

,origin,product_id,price_subtotal,id_x,quantity,product_name,invoice_name,date_invoice,number,id_y,store_name,salesperson_name,sales_team_name,partner_id_num,client_name_inv
39853,False,"[19273, [BCS10061125] GS061A FILTRO COMBUSTIBLE SEPARADOR CASE, GEHL (125 BF7772)]",97800.0,1578874,2.0,"[BCS10061125] GS061A FILTRO COMBUSTIBLE SEPARADOR CASE, GEHL (125 BF7772)",PV6E18345 Mostrador Barranquilla T2/23293,2025-07-24 16:21:37,PV6E18345,428382,SUCURSAL BARRANQUILLA,PAMELA TRUJILLO,SUCURSALES / BARRANQUILLA / Ventas mostrador Barranquilla,5087,[CC13480505] LUIS ALFREDO TORRES RINCON
44413,False,"[18574, [BLS00136125] GS136 FILTRO ACEITE CUMMINS,CASE. (125 B205)]",83000.0,1573665,1.0,"[BLS00136125] GS136 FILTRO ACEITE CUMMINS,CASE. (125 B205)",PV6E18221 Mostrador Barranquilla T2/23200,2025-07-16 21:47:34,PV6E18221,426967,SUCURSAL BARRANQUILLA,PAMELA TRUJILLO,SUCURSALES / BARRANQUILLA / Ventas mostrador Barranquilla,36608,[CO9005048835] J.P.M INGENIERIA Y CONSTRUCCIONES S.A.S
13445,BD1\OUT\60056,"[47436, [DCS00331189] GS331 FILTRO SEPARADOR COMBUSTIBLE - NISSAN (189 GS331)]",103800.0,1610344,6.0,[DCS00331189] GS331 FILTRO SEPARADOR COMBUSTIBLE - NISSAN (189 GS331),FCOT26098,2025-09-10 12:27:39,FCOT26098,436742,PRINCIPAL COTA,WILMER GIL,EXTERNOS,27071,[CC46450716] RUTH ADRIANA LINARES CABRA
50660,False,"[33104, [DAP04020025] DA4020 FILTRO AIRE DONSSON XPEC -CATERPILLAR (025 DA4020)]",48000.0,1566522,1.0,[DAP04020025] DA4020 FILTRO AIRE DONSSON XPEC -CATERPILLAR (025 DA4020),PV7E7600 Mostrador Bucaramanga T1/7768,2025-07-07 21:40:14,PV7E7600,425123,SUCURSAL BUCARAMANGA,EDISSON MIGUEL CIFUENTES OSORIO,SUCURSALES / Sucursal Bucaramanga / MOSTRADOR BM CONTADO,31775,[CO9014710822] INVERSIONES HRS S.A.S
5608,BD6/OUT/49521,"[19382, [BCE00654125] G654 FILTRO COMBUSTIBLE SEPARADOR JCB 320/07057 (125 BF7965)]",577600.0,1619538,8.0,[BCE00654125] G654 FILTRO COMBUSTIBLE SEPARADOR JCB 320/07057 (125 BF7965),FBAQ14372,2025-09-22 22:01:43,FBAQ14372,439191,SUCURSAL BARRANQUILLA,JAIME ALBERTO CAÑON RAMIREZ,SUCURSALES / BARRANQUILLA / EXTERNOS,3177,[CO9006252297] MULTIFILTROS CARTAGENA S.A.S.


In [41]:
df_merged[df_merged["product_name"].str.contains("DA4020", case =False , na=False)].sample(10)

,origin,product_id,price_subtotal,id_x,quantity,product_name,invoice_name,date_invoice,number,id_y,store_name,salesperson_name,sales_team_name,partner_id_num,client_name_inv
10598,False,"[33104, [DAP04020025] DA4020 FILTRO AIRE DONSSON XPEC -CATERPILLAR (025 DA4020)]",48000.0,1613668,1.0,[DAP04020025] DA4020 FILTRO AIRE DONSSON XPEC -CATERPILLAR (025 DA4020),PV7E8180 Mostrador Bucaramanga T1/7976,2025-09-15 13:51:46,PV7E8180,437670,SUCURSAL BUCARAMANGA,EDISSON MIGUEL CIFUENTES OSORIO,SUCURSALES / Sucursal Bucaramanga / MOSTRADOR BM CONTADO,31775,[CO9014710822] INVERSIONES HRS S.A.S
14627,BD6/OUT/49074,"[33104, [DAP04020025] DA4020 FILTRO AIRE DONSSON XPEC -CATERPILLAR (025 DA4020)]",48000.0,1608956,1.0,[DAP04020025] DA4020 FILTRO AIRE DONSSON XPEC -CATERPILLAR (025 DA4020),FBAQ14236,2025-09-08 17:09:26,FBAQ14236,436376,SUCURSAL BARRANQUILLA,MIGUEL ANGEL NIEBLES CERA,SUCURSALES / BARRANQUILLA / EXTERNOS,39686,[CO189395158] JORGE CLAVIJO BUSTOS
41248,BD6/OUT/47821,"[33104, [DAP04020025] DA4020 FILTRO AIRE DONSSON XPEC -CATERPILLAR (025 DA4020)]",43500.0,1577252,1.0,[DAP04020025] DA4020 FILTRO AIRE DONSSON XPEC -CATERPILLAR (025 DA4020),FBAQ13919,2025-07-23 12:42:26,FBAQ13919,427992,SUCURSAL BARRANQUILLA,JAIME ALBERTO CAÑON RAMIREZ,SUCURSALES / BARRANQUILLA / EXTERNOS,2973,[CO9000710689] LUBRICANTES Y FILTROS MEDELLIN S.A.S.
36341,False,"[33104, [DAP04020025] DA4020 FILTRO AIRE DONSSON XPEC -CATERPILLAR (025 DA4020)]",48000.0,1583077,1.0,[DAP04020025] DA4020 FILTRO AIRE DONSSON XPEC -CATERPILLAR (025 DA4020),PV2E41994 Mostrador Calle 6 T1/24709,2025-07-30 21:33:44,PV2E41994,429557,SUCURSAL CALLE 6,MAICOL MORALES RAMIREZ,SUCURSALES / Ventas Mostrador Cll6,3886,[CO8300353171] DIESEL COMUNEROS LTDA
17295,False,"[33104, [DAP04020025] DA4020 FILTRO AIRE DONSSON XPEC -CATERPILLAR (025 DA4020)]",48000.0,1605858,1.0,[DAP04020025] DA4020 FILTRO AIRE DONSSON XPEC -CATERPILLAR (025 DA4020),PV3E19665 Mostrador Valladolid T2/15151,2025-09-03 14:09:03,PV3E19665,435565,SUCURSAL VALLADOLID,SEBAS RODRIGUEZ,SUCURSALES / Ventas Mostrador Valladolid,4874,[CO8305136098] ABC FILTROS LTDA
37694,False,"[23354, [BAE04020125] DA4020 FILTRO AIRE -CATERPILLAR (125 PA4997)]",157200.0,1581479,1.0,[BAE04020125] DA4020 FILTRO AIRE -CATERPILLAR (125 PA4997),PV7E7771 Mostrador Bucaramanga T2/2136,2025-07-29 14:19:38,PV7E7771,429119,SUCURSAL BUCARAMANGA,SANDRA RAMIREZ,SUCURSALES / Sucursal Bucaramanga / MOSTRADOR BM CONTADO,44602,[CO9015971711] GSMINTERNACIONAL.COM SAS.
5230,False,"[33104, [DAP04020025] DA4020 FILTRO AIRE DONSSON XPEC -CATERPILLAR (025 DA4020)]",48000.0,1619952,1.0,[DAP04020025] DA4020 FILTRO AIRE DONSSON XPEC -CATERPILLAR (025 DA4020),PV4E16233 Mostrador Norte T2/13098,2025-09-23 15:40:39,PV4E16233,439294,SUCURSAL NORTE,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES / Ventas Mostrador Norte,7892,[CC79161450] RAFAEL ANTONIO REYES MOLINA
58020,False,"[33104, [DAP04020025] DA4020 FILTRO AIRE DONSSON XPEC -CATERPILLAR (025 DA4020)]",48000.0,1556323,1.0,[DAP04020025] DA4020 FILTRO AIRE DONSSON XPEC -CATERPILLAR (025 DA4020),PV7E7497 Mostrador Bucaramanga T2/1900,2025-06-21 16:23:05,PV7E7497,422668,SUCURSAL BUCARAMANGA,SANDRA RAMIREZ,SUCURSALES / Sucursal Bucaramanga / MOSTRADOR BM CONTADO,31775,[CO9014710822] INVERSIONES HRS S.A.S
33218,BD2\OUT\105558,"[33104, [DAP04020025] DA4020 FILTRO AIRE DONSSON XPEC -CATERPILLAR (025 DA4020)]",48000.0,1586572,1.0,[DAP04020025] DA4020 FILTRO AIRE DONSSON XPEC -CATERPILLAR (025 DA4020),FCL61548,2025-08-06 13:18:23,FCL61548,430619,SUCURSAL CALLE 6,Camila Arguello,SUCURSALES / Ventas Mostrador Cll6,4997,[CO9001808097] TRITURADOS TG S.A.
28897,BD1\OUT\59662,"[33104, [DAP04020025] DA4020 FILTRO AIRE DONSSON XPEC -CATERPILLAR (025 DA4020)]",348000.0,1592340,8.0,[DAP04020025] DA4020 FILTRO AIRE DONSSON XPEC -CATERPILLAR (025 DA4020),FCOT25668,2025-08-13 20:12:49,FCOT25668,431809,PRINCIPAL COTA,ANDRES CORRALES,EXTERNOS,24169,[CC65709095] MARIA VICTORIA GONZALEZ BARRAGAN


# COSTO DEL PRODUCTO

In [ ]:
import xmlrpc.client
import pandas as pd


# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


#COSTO DE LOS PRODUCTOS

campos = models.execute_kw(
    db , uid , password,
    "cs.analisis.costo","fields_get",
    [],

)

df_campos = pd.DataFrame(campos)
#df_campos.info()


productos_costo = models.execute_kw(
    db , uid , password,
    "cs.analisis.costo","search_read",
    [[]],
    {"fields":["producto_costo_unitario","producto_id"]}


)

# Convertir en df
df_productos_costo = pd.DataFrame(productos_costo)


# Separar lista anidada
df_productos_costo[["product_id_num","product_name"]] = (df_productos_costo["producto_id"].apply(pd.Series))

#Agregar columna estandarizada
df_productos_costo["product_name"] = df_productos_costo["product_name"].str.strip()

#DF normalizado
df_productos_costo = df_productos_costo[["product_name","producto_costo_unitario"]]

df_productos_costo.to_parquet("costo_productos.parquet")


# ANALISIS COSTOS

In [12]:
import xmlrpc.client
import pandas as pd


# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


#COSTO DE LOS PRODUCTOS

productos_ema = models.execute_kw(
    db, uid, password,
    "cs.analisis.costo", "search_read",[],
    #[
    #    [["semana","=",36]]
    #],
    {"fields": ["sucursal_id","product_name","ano","semana","ema","producto_costo_unitario","acumulado_combinado","clasificacion","almacen_id" ]}
)

df = pd.DataFrame(productos_ema)

def extract_id(val):
    if isinstance(val, (list, tuple)) and len(val) > 0:
        return val[0]
    return None

def extract_name(val):
    if isinstance(val, (list, tuple)) and len(val) > 1:
        return val[1]
    return str(val)

df["store_id_num"] = df["sucursal_id"].apply(extract_id)
df["store_name"]   = df["sucursal_id"].apply(extract_name)

df["almacen_id_num"] = df["almacen_id"].apply(extract_id)
df["almacen_name"]   = df["almacen_id"].apply(extract_name)

df.to_csv("real_ema.csv")

#timpo aprox  1 minutos

# VENTAS PERDIDAS

In [2]:
import xmlrpc.client
from datetime import date, timedelta
import pandas as pd

# ===============================
# 1. Conexión con Odoo
# ===============================

ODOO_URL = "https://donsson.com"
ODOO_DB = "Donsson_produccion"
ODOO_USERNAME = "juan.cano@donsson.com"
ODOO_PASSWORD = "1000285668"

# Autenticación
common = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/common")
uid = common.authenticate(ODOO_DB, ODOO_USERNAME, ODOO_PASSWORD, {})
if not uid:
    print("Error de autenticación. Verifica tus credenciales.")
    exit()
models = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/object")

# ===============================
# 2. Definir rango de semanas
# ===============================

# Número de semanas que quieres descargar (puedes cambiarlo)
NUM_SEMANAS = 52  

# Fecha de hoy
hoy = date.today()
# Fecha de inicio según número de semanas atrás
fecha_inicio = hoy - timedelta(weeks=NUM_SEMANAS)

FECHA_INICIO = fecha_inicio.strftime("%Y-%m-%d")
FECHA_FIN = hoy.strftime("%Y-%m-%d")

print(f"📅 Descargando registros desde {FECHA_INICIO} hasta {FECHA_FIN}")

# ===============================
# 3. Leer ventas perdidas
# ===============================

lost_sales_ids = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "ventas.perdidas", "search",
    [[["fecha", ">=", FECHA_INICIO], ["fecha", "<=", FECHA_FIN], ["almacenamiento_tipo", "=", "agotado"]]]
)

lost_sales_data = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "ventas.perdidas", "read",
    [lost_sales_ids],
    {"fields": ["origen", "store_id", "cliente_id", "fecha", "product_ref",
                "cantidad", "cantidad_existencia", "cantidad_reservada",
                "almacenamiento_tipo"]}
)

df = pd.DataFrame(lost_sales_data)

# ===============================
# 4. Normalizar columnas
# ===============================

for col in ["store_id", "cliente_id"]:
    df[col.replace('_id', '_name')] = df[col].apply(
        lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else str(x) if x else None
    )
    df[col + "_num"] = df[col].apply(
        lambda x: x[0] if isinstance(x, (list, tuple)) and len(x) > 0 else None
    )

# Eliminar columnas originales
df = df.drop(columns=["id", "store_id", "cliente_id"], errors='ignore')

# ===============================
# 4.1. Forzar tipos de columnas antes de exportar
# ===============================
for col in df.columns:
    if col.endswith("_num"):
        df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")
    else:
        df[col] = df[col].astype("string")

print(df.dtypes)


# ===============================
# 5. Exportar a Parquet con nombre dinámico
# ===============================

file_name = f"ventas_perdidas_{FECHA_INICIO}_a_{FECHA_FIN}.parquet"

try:
    df.to_parquet(file_name, index=False, engine="pyarrow", compression="snappy")
    print(f"✅ Exportación exitosa con pyarrow: {file_name}")
except ImportError as e:
    print("⚠️ Módulo pyarrow no encontrado, probando con fastparquet:", e)
    try:
        df.to_parquet(file_name, index=False, engine="fastparquet", compression="snappy")
        print(f"✅ Exportación exitosa con fastparquet: {file_name}")
    except ImportError as e:
        print("❌ Instala 'pyarrow' o 'fastparquet' para exportar.")

print("\n")
print(f"El proceso ha terminado. Se extrajeron {len(df)} registros de ventas perdidas.")
print("Las columnas del DataFrame final son:", list(df.columns))



    ##TARDA MAS O MENOS 1 MINUTOS TODO 2025


📅 Descargando registros desde 2024-10-02 hasta 2025-10-01
cantidad_reservada     string[python]
product_ref            string[python]
cantidad               string[python]
almacenamiento_tipo    string[python]
fecha                  string[python]
cantidad_existencia    string[python]
origen                 string[python]
store_name             string[python]
store_id_num                    Int64
cliente_name           string[python]
cliente_id_num                  Int64
dtype: object
✅ Exportación exitosa con pyarrow: ventas_perdidas_2024-10-02_a_2025-10-01.parquet


El proceso ha terminado. Se extrajeron 68549 registros de ventas perdidas.
Las columnas del DataFrame final son: ['cantidad_reservada', 'product_ref', 'cantidad', 'almacenamiento_tipo', 'fecha', 'cantidad_existencia', 'origen', 'store_name', 'store_id_num', 'cliente_name', 'cliente_id_num']


# MOTOR BUSQUEDA

In [3]:
import xmlrpc.client
import pandas as pd

# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# ===============================
# Ver campos disponible
# ===============================
fields = models.execute_kw(
    db, uid, password,
    "res.partner", "fields_get", 
    []
)

for field, info in sorted(fields.items(), key=lambda x: x[1].get("string", "")):
    print(f"{field:25} | {info.get('type')} | {info.get('string')}")



claim_count               | integer | # Claims
issue_count               | integer | # Issues
meeting_count             | integer | # Meetings
supplier_invoice_count    | integer | # Supplier Invoices
task_count                | integer | # Tasks
purchase_order_count      | integer | # of Purchase Order
sale_order_count          | integer | # of Sales Order
property_account_payable  | many2one | Account Payable
property_account_prepayable | many2one | Account Payable (Prepayment)
property_account_receivable | many2one | Account Receivable
property_account_prereceivable | many2one | Account Receivable (Prepayment)
active                    | boolean | Active
activity_economic_id      | many2one | Actividad economica
type                      | selection | Address Type
payment_amount_due        | float | Amount Due
payment_amount_overdue    | float | Amount Overdue
answers_ids               | many2many | Answers
is_autorizado             | boolean | Autorizado
bank_ids                  |

In [ ]:
import xmlrpc.client
import pandas as pd

# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# 1) Traer nóminas (slips) solo de 2025
# ========================================
slips = models.execute_kw(
    db, uid, password,
    "hr.payslip", "search_read",
    [[["date_from", ">=", "2024-01-01"], ["date_to", "<=", "2025-12-31"]]],
    {"fields": ["id", "employee_id", "name", "date_from", "date_to"]}
)
df_slips = pd.DataFrame(slips)


# ========================================
# 2) Traer líneas de nómina (solo de los slips obtenidos)
# ========================================
slip_ids = df_slips["id"].tolist()
lines = models.execute_kw(
    db, uid, password,
    "hr.payslip.line", "search_read",
    [[["slip_id", "in", slip_ids]]],
    {"fields": ["id", "slip_id", "employee_id", "salary_rule_id", "total"]}
)
df_lines = pd.DataFrame(lines)

# ===============================
# 3) Expandir campos Many2one (id, name)
# ===============================
def extract_name(val):
    if isinstance(val, list) and len(val) == 2:
        return val[1]
    return None

def extract_id(val):
    if isinstance(val, list) and len(val) == 2:
        return val[0]
    return None

df_lines["employee_name"] = df_lines["employee_id"].apply(extract_name)
df_lines["slip_name"]     = df_lines["slip_id"].apply(extract_name)
df_lines["rule_name"]     = df_lines["salary_rule_id"].apply(extract_name)

# 🔑 Extraer ID de slip_id para poder hacer el merge
df_lines["slip_id_only"] = df_lines["slip_id"].apply(extract_id)

# ===============================
# 4) Unir líneas con fechas de slip
# ===============================
df_slips["employee_name"] = df_slips["employee_id"].apply(extract_name)

df = pd.merge(
    df_lines,
    df_slips[["id", "employee_name", "date_from", "date_to"]],
    left_on="slip_id_only",
    right_on="id",
    how="left",
    suffixes=("", "_slip")
)

# ===============================
# 5) Filtrar reglas relevantes
# ===============================
df_filtro = df[df["rule_name"].isin(
    ["Salario Básico", "Salario Mínimo", "Salario Neto"]
)].copy()

# ===============================
# 6) Agregar columna período (Año-Mes)
# ===============================
df_filtro["periodo"] = pd.to_datetime(df_filtro["date_from"]).dt.to_period("M")

# ===============================
# 7) Pivotear para resumir salarios
# ===============================
tabla = df_filtro.pivot_table(
    index=["employee_name", "periodo"],
    columns="rule_name",
    values="total",
    aggfunc="sum",
    fill_value=0
).reset_index()

# ===============================
# 8) Renombrar columnas
# ===============================
tabla = tabla.rename(columns={
    "Salario Básico": "salario_basico",
    "Salario Mínimo": "salario_minimo",
    "Salario Neto": "salario_neto"
})

# ===============================
# 9) Calcular salario base = máximo entre básico y mínimo
# ===============================
tabla["salario_base"] = tabla[["salario_basico", "salario_minimo"]].max(axis=1)

# ===============================
# 10) Tabla final
# ===============================
resultado = tabla[["employee_name", "periodo", "salario_base", "salario_neto"]]

print("✅ Resumen mensual de salarios")

#aprox2min

In [ ]:
r =resultado.sort_values(["salario_neto"],ascending=False)
r = r[r["periodo"]=="2025-09"]
r.head(30)